Connect to drive

In [ ]:
from google.colab import drive
drive.mount('/root/gdrive', force_remount=True)

Mounted at /root/gdrive


In [ ]:
%cd /root/gdrive/MyDrive/Bangla Newspaper Classification

/root/gdrive/MyDrive/Bangla Newspaper Classification


Import libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from collections import Counter

Load data

In [ ]:
import json

with open('data/data.json', encoding='utf-8') as fh:
    data = json.load(fh)
print(data[0])

{'author': 'গাজীপুর প্রতিনিধি', 'category': 'bangladesh', 'category_bn': 'বাংলাদেশ', 'published_date': '০৪ জুলাই ২০১৩, ২৩:২৬', 'modification_date': '০৪ জুলাই ২০১৩, ২৩:২৭', 'tag': ['গাজীপুর'], 'comment_count': 0, 'title': 'কালিয়াকৈরে টিফিন খেয়ে ৫০০ শ্রমিক অসুস্থ, বিক্ষোভ', 'url': 'http://www.prothom-alo.com/bangladesh/article/19030', 'content': 'গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায় আজ বৃহস্পতিবার রাতের টিফিন খেয়ে একটি পোশাক কারখানার ৫০০ শ্রমিক অসুস্থ হয়ে পড়েছেন। এ ঘটনায় বিক্ষোভ করেছেন ওই কারখানার শ্রমিকেরা।সফিপুর মডার্ন হাসপাতালের জরুরি বিভাগের চিকিত্সক আল আমিন প্রথম আলো ডটকমকে বলেন, খাদ্যে বিষক্রিয়ায় তাঁরা (শ্রমিকেরা) অসুস্থ হয়ে পড়েছেন। এতে আতঙ্কিত হওয়ার কিছু নেই। অসুস্থদের চিকিত্সা দেওয়া হয়েছে।কারখানার শ্রমিক ও পুলিশ সূত্রে জানা যায়, উপজেলার তেলিরচালা এলাকার সেজাদ সোয়েটার লিমিটেড কারখানার শ্রমিকদের আজ রাত সাড়ে সাতটার দিকে টিফিন দেওয়া হয়। টিফিনে ছিল ডিম, রুটি, পেটিস ও কলা। টিফিন খেয়ে শ্রমিকেরা যথারীতি কাজে যোগ দেন। ওই টিফিন খাওয়ার প্রায় এক ঘণ্টা পর রাত সা

Trim data for test run

In [ ]:
data = data[:50000]

Visualize data

In [ ]:
print("Total Rows:", len(data))
print("Total Cols:", len(data[0]))

Total Rows: 80000
Total Cols: 10


Show all category

In [ ]:
print(type(data))
print("All columns:")
for col in data[0]:
  print(col)

<class 'list'>
All columns:
author
category
category_bn
published_date
modification_date
tag
comment_count
title
url
content


Retrieve all categories

In [ ]:
all_categories = set([sample['category'] for sample in data])
print(all_categories)
print(len(all_categories))

{'international', 'durporobash', '-1', 'events', 'opinion', 'entertainment', 'education', 'life-style', 'we-are', 'onnoalo', 'facebook', 'pachmisheli', 'sports', 'economy', 'roshalo', 'special-supplement', 'AskEditor', 'bangladesh', 'kishoralo', 'technology'}
20


Sorting based on category count

In [ ]:
category_count_list = []
count = 0
for cat in all_categories:
  count = 0
  for row in data:
    if row['category'] == cat:
      count += 1
  category_count_list.append({'category': cat, 'count': count})
print(category_count_list)

from operator import itemgetter
sorted_cat_count_list = []
# sorted_cat_count_list = sorted(category_count_list, key=lambda x: x.count, reverse=True)
category_count_list.sort(key = itemgetter('count'), reverse=True)
print(category_count_list)

[{'category': 'international', 'count': 5793}, {'category': 'durporobash', 'count': 1}, {'category': '-1', 'count': 27}, {'category': 'events', 'count': 2}, {'category': 'opinion', 'count': 2542}, {'category': 'entertainment', 'count': 5476}, {'category': 'education', 'count': 2509}, {'category': 'life-style', 'count': 1837}, {'category': 'we-are', 'count': 1149}, {'category': 'onnoalo', 'count': 442}, {'category': 'facebook', 'count': 10}, {'category': 'pachmisheli', 'count': 652}, {'category': 'sports', 'count': 9558}, {'category': 'economy', 'count': 2984}, {'category': 'roshalo', 'count': 551}, {'category': 'special-supplement', 'count': 152}, {'category': 'AskEditor', 'count': 1}, {'category': 'bangladesh', 'count': 43003}, {'category': 'kishoralo', 'count': 197}, {'category': 'technology', 'count': 3114}]
[{'category': 'bangladesh', 'count': 43003}, {'category': 'sports', 'count': 9558}, {'category': 'international', 'count': 5793}, {'category': 'entertainment', 'count': 5476}, {

Discard low count catagories

In [ ]:
selected_cats = []

for p in category_count_list:
    if p['count'] > 1000:
        selected_cats.append(p['category'])
print(selected_cats)
print(len(selected_cats))

['bangladesh', 'sports', 'international', 'entertainment', 'technology', 'economy', 'opinion', 'education', 'life-style', 'we-are']
10


Prepare data X, Y

In [ ]:
x_text = []
y_label = []

for row in data:
    if row['category'] in selected_cats:
        y_label.append(row['category'])
        x_text.append(row['content'])
print(len(x_text),len(y_label))

77965 77965


Split data into Train & test set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_text,y_label, stratify=y_label)


Import neccessary libraries

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
!pip install tensorflow-text
import tensorflow_text as text

     |████████████████████████████████| 4.9 MB 5.4 MB/s 
     |████████████████████████████████| 462 kB 43.1 MB/s 


import multi-bert preprocessor & encoder from tensorflow hub

In [ ]:
bert_preprocess = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3')
bert_encoder = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/4')

Build classifier model

In [ ]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)
# Neural network layers
net = tf.keras.layers.Dense(20, activation='relu')(outputs['pooled_output'])
net = tf.keras.layers.Dropout(0.2)(net)
# net = tf.keras.layers.Dense(32, activation='relu')(net)
# net = tf.keras.layers.Dropout(0.2)(net)
# net = tf.keras.layers.Dense(16,  activation='relu')(net)
# net = tf.keras.layers.Dropout(0.1)(net)
out = tf.keras.layers.Dense(10, activation='softmax', name="output")(net)
# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [out])

One hot encoding of Y

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
def make_ohe(y_label):
  # Label Encode
  encoder = LabelEncoder()
  class_labels = encoder.fit_transform(y_label)
  print(class_labels,len(class_labels),class_labels.shape)
  print(set(class_labels))

  #One hot Encode
  encoder = OneHotEncoder(sparse=False)
  class_labels = class_labels.reshape((class_labels.shape[0], 1))
  y_ohe = encoder.fit_transform(class_labels)
  print(y_ohe,y_ohe.shape)
  return y_ohe

In [ ]:
y_train = make_ohe(y_train)
print(type(y_train))

[0 0 0 ... 7 0 3] 58473 (58473,)
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] (58473, 10)
<class 'numpy.ndarray'>


In [ ]:
y_test = make_ohe(y_test)
print(type(y_test))

[0 0 0 ... 2 0 9] 19492 (19492,)
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]] (19492, 10)
<class 'numpy.ndarray'>


In [ ]:
X_train = np.asarray(X_train)
print(type(X_train))

<class 'numpy.ndarray'>


Train

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('mbert_80k.h5', monitor='loss', save_best_only=True, verbose=1)
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=15, verbose=1)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
train_history = model.fit(X_train, y_train, epochs=10,callbacks=[checkpoint, earlystopping], batch_size = 32)

Epoch 1/10
1828/1828 [==============================] - ETA: 0s - loss: 1.1397 - accuracy: 0.6576
Epoch 1: loss improved from inf to 1.13969, saving model to mbert_80k.h5


1828/1828 [==============================] - 1320s 713ms/step - loss: 1.1397 - accuracy: 0.6576
Epoch 2/10
1828/1828 [==============================] - ETA: 0s - loss: 0.8658 - accuracy: 0.7390
Epoch 2: loss improved from 1.13969 to 0.86581, saving model to mbert_80k.h5


1828/1828 [==============================] - 1295s 708ms/step - loss: 0.8658 - accuracy: 0.7390
Epoch 3/10
1828/1828 [==============================] - ETA: 0s - loss: 0.7924 - accuracy: 0.7546
Epoch 3: loss improved from 0.86581 to 0.79244, saving model to mbert_80k.h5


1828/1828 [==============================] - 1294s 708ms/step - loss: 0.7924 - accuracy: 0.7546
Epoch 4/10
1828/1828 [==============================] - ETA: 0s - loss: 0.7525 - accuracy: 0.7618
Epoch 4: loss improved from 0.79244 to 0.75252, saving model to mbert_80k.h5


1828/1828 [==============================] - 1293s 707ms/step - loss: 0.7525 - accuracy: 0.7618
Epoch 5/10
1828/1828 [==============================] - ETA: 0s - loss: 0.7269 - accuracy: 0.7679
Epoch 5: loss improved from 0.75252 to 0.72686, saving model to mbert_80k.h5


...

1828/1828 [==============================] - 1293s 707ms/step - loss: 0.7100 - accuracy: 0.7728
Epoch 7/10
1828/1828 [==============================] - ETA: 0s - loss: 0.6982 - accuracy: 0.7785
Epoch 7: loss improved from 0.71001 to 0.69820, saving model to mbert_80k.h5


1828/1828 [==============================] - 1292s 707ms/step - loss: 0.6982 - accuracy: 0.7785
Epoch 8/10
1828/1828 [==============================] - ETA: 0s - loss: 0.6858 - accuracy: 0.7836
Epoch 8: loss improved from 0.69820 to 0.68581, saving model to mbert_80k.h5


1828/1828 [==============================] - 1288s 705ms/step - loss: 0.6858 - accuracy: 0.7836
Epoch 9/10
1828/1828 [==============================] - ETA: 0s - loss: 0.6769 - accuracy: 0.7849
Epoch 9: loss improved from 0.68581 to 0.67694, saving model to mbert_80k.h5


1828/1828 [==============================] - 1288s 704ms/step - loss: 0.6769 - accuracy: 0.7849
Epoch 10/10
1828/1828 [==============================] - ETA: 0s - loss: 0.6688 - accuracy: 0.7883
Epoch 10: loss improved from 0.67694 to 0.66875, saving model to mbert_80k.h5


1828/1828 [==============================] - 1293s 707ms/step - loss: 0.6688 - accuracy: 0.7883


Save model

In [ ]:
model.save("mbert_80k.h5")

In [ ]:
# model = tf.keras.models.load_model("mbert_80k.h5")
saved_model = tf.keras.models.load_model("mbert_80k.h5",custom_objects={'KerasLayer':hub.KerasLayer})

In [ ]:
model = saved_model 

Train data accuracy evaluation

In [ ]:
y_predicted_train = model.predict(X_train)
y_out_train = []
y_out_pred_train = []
true_count = 0
for i in range(len(y_predicted_train)):
  y_out_train.append(np.argmax(y_train[i]))
  y_out_pred_train.append(np.argmax(y_predicted_train[i]))
  if (y_out_train[i] == y_out_pred_train[i]):
    true_count += 1
# print(np.shape(y_out),np.shape(y_label))

accuracy = true_count/len(y_train)
print(accuracy)
# print(y_out)

0.6740204880885194


Test data accuracy evaluation

In [ ]:
y_predicted = model.predict(X_test)
y_out = []
y_out_pred = []
true_count = 0
for i in range(len(y_predicted)):
  y_out.append(np.argmax(y_test[i]))
  y_out_pred.append(np.argmax(y_predicted[i]))
  if (y_out[i] == y_out_pred[i]):
    true_count += 1
print(np.shape(y_out),np.shape(y_label))

accuracy = true_count/len(y_test)
print(accuracy)
# print(y_out)